In [10]:
from azureml.core import Workspace, Run
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice

Connect to the Azure Machine Learning Services Workspace using information in config.json. If not yet authenticated, you will be redirected to the authentication page.

In [2]:
ws = Workspace.from_config()

Define prediction runtime container image.

In [12]:
myenv = CondaDependencies()
myenv.add_conda_package("keras")

with open("conda-config.yml","w") as f:
    f.write(myenv.serialize_to_string())

image_config = ContainerImage.image_configuration(execution_script="predict.py", 
                                                  runtime="python",
                                                  conda_file="myenv.yml")

ERROR - Execution script predict.py doesn't exist



WebserviceException: Execution script predict.py doesn't exist